In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# 라이브러리 선언
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

In [7]:
# OpenAI 객체를 생성합니다.
model = ChatOpenAI(temperature=0, model_name="gpt-4.1-mini")

In [24]:
# 원하는 데이터 구조를 정의합니다.
class Filter(BaseModel):
    SALE_PRC: str = Field(description="가격 범위. 예를 들어 100만원 이상 120만원 이하이면, '$gte': 1000000, '$lte': 1200000")
    BRND_NM: str = Field(description="브랜드명")
    ARTC_NM: str = Field(description="품목")
    CATEGORY_NM: str = Field(description="카테고리명")
    FEATURES: str = Field(description="주요기능")

# {'SALE_PRC': {'$gte': 1000000, '$lte': 1200000}}

In [25]:
# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Filter)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"SALE_PRC": {"description": "가격 범위. 예를 들어 100만원 이상 120만원 이하이면, '$gte': 1000000, '$lte': 1200000", "title": "Sale Prc", "type": "string"}, "BRND_NM": {"description": "브랜드명", "title": "Brnd Nm", "type": "string"}, "ARTC_NM": {"description": "품목", "title": "Artc Nm", "type": "string"}, "CATEGORY_NM": {"description": "카테고리명", "title": "Category Nm", "type": "string"}, "FEATURES": {"description": "주요기능", "title": "Features", "type": "string"}}, "required": ["SALE_PRC", "BRND_NM", "ARTC_NM", "CATEGORY_NM", "FEATURES"]}
```


In [26]:

# 프롬프트를 생성합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 가전전문 e커머스 검색 AI 어시스턴트 입니다. 사용자의 자연어 검색어를 받아, 그 의도를 분석하여 검색 필터 조건을 완성합니다."),
        ("user", "#Format: {format_instructions}\n\n#Question: {query}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 체인을 구성합니다.
chain = prompt | model | parser

In [27]:
# 체인을 호출하여 쿼리 실행
query = "5만원 이상 블루투스 마우스"
answer = chain.invoke({"query": query})
print(f"{query}: {answer}")

query = "4인 가족 냉장고 추천"
answer = chain.invoke({"query": query})
print(f"{query}: {answer}")

query = "원룸용 세탁기"
answer = chain.invoke({"query": query})
print(f"{query}: {answer}")

query = "게이밍용 고성능 모니터"
answer = chain.invoke({"query": query})
print(f"{query}: {answer}")

query = "신혼부부용 가전세트"
answer = chain.invoke({"query": query})
print(f"{query}: {answer}")

query = "국산 로봇청소기 추천"
answer = chain.invoke({"query": query})
print(f"{query}: {answer}")

query = "4K HDR 지원 55인치 TV"
answer = chain.invoke({"query": query})
print(f"{query}: {answer}")

query = "20만원대 스팀 다리미"
answer = chain.invoke({"query": query})
print(f"{query}: {answer}")

query = "인덕션 3구 전기레인지"
answer = chain.invoke({"query": query})
print(f"{query}: {answer}")

query = "조용하고 전기요금 적게 나오는 에어컨"
answer = chain.invoke({"query": query})
print(f"{query}: {answer}")

query = "자동급수 가능한 로봇청소기"
answer = chain.invoke({"query": query})
print(f"{query}: {answer}")




5만원 이상 블루투스 마우스: {'SALE_PRC': '$gte: 50000', 'BRND_NM': '', 'ARTC_NM': '마우스', 'CATEGORY_NM': '블루투스 마우스', 'FEATURES': '블루투스'}
4인 가족 냉장고 추천: {'SALE_PRC': '', 'BRND_NM': '', 'ARTC_NM': '냉장고', 'CATEGORY_NM': '가전제품', 'FEATURES': '4인 가족'}
원룸용 세탁기: {'SALE_PRC': '', 'BRND_NM': '', 'ARTC_NM': '세탁기', 'CATEGORY_NM': '원룸용', 'FEATURES': ''}
게이밍용 고성능 모니터: {'SALE_PRC': '', 'BRND_NM': '', 'ARTC_NM': '모니터', 'CATEGORY_NM': '컴퓨터/노트북/주변기기', 'FEATURES': '게이밍, 고성능'}
신혼부부용 가전세트: {'SALE_PRC': '', 'BRND_NM': '', 'ARTC_NM': '가전세트', 'CATEGORY_NM': '신혼부부용', 'FEATURES': ''}
국산 로봇청소기 추천: {'SALE_PRC': '', 'BRND_NM': '국산', 'ARTC_NM': '로봇청소기', 'CATEGORY_NM': '청소기', 'FEATURES': ''}
4K HDR 지원 55인치 TV: {'SALE_PRC': '', 'BRND_NM': '', 'ARTC_NM': 'TV', 'CATEGORY_NM': 'TV', 'FEATURES': '4K HDR, 55인치'}
20만원대 스팀 다리미: {'SALE_PRC': '$gte:200000, $lt:300000', 'BRND_NM': '', 'ARTC_NM': '다리미', 'CATEGORY_NM': '스팀다리미', 'FEATURES': '스팀'}
인덕션 3구 전기레인지: {'SALE_PRC': '', 'BRND_NM': '', 'ARTC_NM': '전기레인지', 'CATEGORY_NM': '인덕션', 'FEATURES